In [1]:
import polars as pl

In [2]:
df = pl.read_parquet("data/*.parquet")
df.head()

created_utc,title,upvotes,num_comments,upvote_ratio,is_created_from_ads_ui,total_awards_received,num_reports,image_url,is_video,sub_reddit,created_at,meme_text
i64,str,i64,i64,f64,bool,i64,null,str,bool,str,datetime[μs],null
1768284466,"""Yi Long Musk""",3233,31,0.98,false,0,null,"""https://i.redd.it/cii85evk52dg…",false,"""dankmemes""",2026-01-13 06:07:46,null
1768283328,"""Thank me mf""",1781,19,0.99,false,0,null,"""https://i.redd.it/nrkx4kv522dg…",false,"""dankmemes""",2026-01-13 05:48:48,null
1768285559,"""What you mean I can’t date the…",1250,52,0.98,false,0,null,"""https://i.redd.it/rzk6x02e82dg…",false,"""dankmemes""",2026-01-13 06:25:59,null
1768294538,"""I wonder""",776,17,0.93,false,0,null,"""https://i.redd.it/qngxi41jz2dg…",false,"""dankmemes""",2026-01-13 08:55:38,null
1768255736,"""What a way to start the day""",6213,68,0.98,false,0,null,"""https://i.redd.it/c35e47pqrzcg…",false,"""dankmemes""",2026-01-12 22:08:56,null


In [3]:
df.shape

(300, 13)

It would be wise to label enginner our variable here. And them check how well of a job we did, after we evaluate it with spearmasn correlation we can proceed

In [4]:

import time
now = time.time()

# Label enginnering
df = (
    df.with_columns(
        ((now - pl.col("created_utc")) / 3600)
        .clip(lower_bound=0)
        .alias("age_hours")
    )
    .with_columns(
        (pl.col("upvotes") + 2.0 * pl.col("num_comments"))
        .alias("engagement")
    )
    .with_columns(
        
        (pl.col("engagement") / (pl.col("age_hours") + 1) ** 0.5)
        .alias("engagement_score")
    )
)


In [5]:
# Sanity check lets see if these memes are firew
with pl.Config(fmt_str_lengths=50):
    print(df.sort(by="engagement_score",descending=True)["image_url"][0:5])

shape: (5,)
Series: 'image_url' [str]
[
	"https://i.redd.it/9hres7v1b3dg1.jpeg"
	"https://i.redd.it/v3ce0xf89vcg1.jpeg"
	"https://i.redd.it/55yi8hqcqqcg1.jpeg"
	"https://i.redd.it/j9amyo8le8cg1.jpeg"
	"https://i.redd.it/kciu8jp5vwcg1.jpeg"
]


In [ ]:
cols = [
    "upvote_ratio",
    "upvotes",
    "num_comments",
    "total_awards_received",
    "engagement",
    "age_hours",
    "num_reports"
]

# Ideally our age hours variable here, should be negatively correlated. To value new memes more, which is essential for a ranker
df.select([
    pl.corr(col, "engagement_score", method="spearman").alias(col)
    for col in cols
])

upvote_ratio,upvotes,num_comments,total_awards_received,engagement,age_hours,num_reports
f64,f64,f64,f64,f64,f64,f64
0.531248,0.85955,0.662046,NaN,0.865845,-0.18904,NaN


In [ ]:
df.group_by("sub_reddit").agg(
    pl.mean("engagement_score").alias("mean_score"),
    pl.len().alias("n_posts"),
)


sub_reddit,mean_score,n_posts
str,f64,u32
"""memes""",141.26531,500
"""Memes_Of_The_Dank""",7.812128,500
"""dankmemes""",112.784072,500
